# Import Library

In [1]:
import os
import sys
import numpy as np
import pandas as pd

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)
    from models.ts_forecaster import TimeSeriesForecaster
    from models.ts_preprocess import generate_future_dataframe

# Load Dataset

In [2]:
df = pd.read_csv("sample_data.csv")
df = df.set_index("date")
df

,y
date,
2019-01-03,19607
2019-01-04,21350
2019-01-05,20176
2019-01-06,22000
2019-01-07,22000
...,...
2021-09-26,8000
2021-09-27,8000
2021-09-28,10714


# Define Steps

In [3]:
steps = 3
df_train = df.iloc[:-steps, :].copy()
df_train.index.max()

'2021-09-27'

In [4]:
df_future = generate_future_dataframe(cutoff="2021-09-28", freq="D", steps=steps)
df_future = df_future.set_index("date")

# Forecast

In [5]:
tsf = TimeSeriesForecaster()

## ARIMA

In [6]:
tsf.train(algo="arima", target="y", df=df_train)
tsf.forecast(df_future=df_future, steps=steps, freq="D", alpha=0.2)

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  999
Model:               SARIMAX(0, 1, 5)   Log Likelihood              -11035.792
Date:                Sat, 04 Feb 2023   AIC                          22083.584
Time:                        22:04:43   BIC                          22113.019
Sample:                    01-03-2019   HQIC                         22094.772
                         - 09-27-2021                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.8851      0.016    -56.889      0.000      -0.916      -0.855
ma.L2         -0.0403      0.032     -1.260      0.208      -0.103       0.022
ma.L3         -0.0194      0.046     -0.419      0.6

In [7]:
tsf.df_fcst

,date,yhat,yhat_lower,yhat_upper
0,2021-09-28,8579.444318,-11463.708178,28622.596813
1,2021-09-29,8598.958981,-11576.079444,28773.997406
2,2021-09-30,8592.879621,-11637.539379,28823.298620


## Prophet

In [8]:
tsf.train(algo="prophet", target="y", df=df_train)
tsf.forecast(df_future=df_future, steps=steps, freq="D")

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/Users/hwangyungwan/miniforge3/envs/ds_p39/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -3.57389
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2696.89    0.00375308       192.199           1           1      123   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2703.56    0.00271891       124.415      0.4607           1      255   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     266       2704.22   0.000128027       123.532    1.13e-06       0.001      392  LS failed, Hessian reset 
     299       2704.44   0.000379943       54.4516           1           1      431   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     331       2704.62   0.000401623        69.998   5.836e-06       0.001      511  LS failed, Hessian reset 
     399       2704.65   9.49532e-06       40.8383      0.2384           1      608   
    Iter      log pro

/Users/hwangyungwan/miniforge3/envs/ds_p39/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/hwangyungwan/miniforge3/envs/ds_p39/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


In [9]:
tsf.df_fcst

,date,yhat,yhat_lower,yhat_upper
0,2021-09-28,6927.427759,-10714.093710,26623.455536
1,2021-09-29,8835.895799,-9343.668023,29007.721357
2,2021-09-30,5479.991107,-13132.922272,24261.494212
